In [0]:
dbutils.widgets.text("table", "")

In [0]:
v=dbutils.widgets.get("table")
print(v)

sample6.json


In [0]:
input_path= "/Volumes/volume/default/bronze"

In [0]:
df=spark.read.format("json").option("multiline","true").load(f"{input_path}/{v}")


In [0]:
df.show()

+--------------------+
|              people|
+--------------------+
|[{28, Joe, male, ...|
+--------------------+



In [0]:
from pyspark.sql.types import StructType, StructField, StringType, LongType, ArrayType
from pyspark.sql.functions import from_json, col, explode

df_people = df.select(explode(col("people")).alias("person"))

# Flatten the struct into columns
df_flat = df_people.select("person.*")

df_flat.show(truncate=False)


+---+---------+------+--------+------------------------------------+
|age|firstName|gender|lastName|number                              |
+---+---------+------+--------+------------------------------------+
|28 |Joe      |male  |Jackson |[7349282382, 7340282382, 7349282386]|
|32 |James    |male  |Smith   |[5678568567, 5678568568]            |
|24 |Emily    |female|Jones   |[5678568567, 456754675]             |
|24 |Swetha   |female|Jones   |[456754675, 5678568567]             |
|24 |usha     |female|Jones   |[456754675, 4567546789, 456754975]  |
+---+---------+------+--------+------------------------------------+



In [0]:
df_flat=df_flat.withColumn("pd",explode(col("number")))

In [0]:
df_flat=df_flat.drop("number")
df_flat.show()

+---+---------+------+--------+----------+
|age|firstName|gender|lastName|        pd|
+---+---------+------+--------+----------+
| 28|      Joe|  male| Jackson|7349282382|
| 28|      Joe|  male| Jackson|7340282382|
| 28|      Joe|  male| Jackson|7349282386|
| 32|    James|  male|   Smith|5678568567|
| 32|    James|  male|   Smith|5678568568|
| 24|    Emily|female|   Jones|5678568567|
| 24|    Emily|female|   Jones| 456754675|
| 24|   Swetha|female|   Jones| 456754675|
| 24|   Swetha|female|   Jones|5678568567|
| 24|     usha|female|   Jones| 456754675|
| 24|     usha|female|   Jones|4567546789|
| 24|     usha|female|   Jones| 456754975|
+---+---------+------+--------+----------+



In [0]:
output_path = "/Volumes/volume/default/silver"

In [0]:
df_flat.write.format("parquet").mode('append').save(f"{output_path}/flatten")

In [0]:
df_flat.show()

+---+---------+------+--------+----------+
|age|firstName|gender|lastName|    number|
+---+---------+------+--------+----------+
| 28|      Joe|  male| Jackson|7349282382|
| 32|    James|  male|   Smith|5678568567|
| 24|    Emily|female|   Jones| 456754675|
+---+---------+------+--------+----------+



from pyspark.sql.types import StructType, StructField, StringType, LongType, ArrayType

p_schema = StructType([
    StructField("age", LongType(), True),
    StructField("firstName", StringType(), True),
    StructField("gender", StringType(), True),
    StructField("lastName", StringType(), True),
    StructField("number", StringType(), True)
])
schema = StructType([
    StructField("people", ArrayType(p_schema), True)
])

